# Attaching streets and discharge intersects to beach data

Streets and discharge intersects are calculated using QGIS, the output is a .csv file that has the corresponding location name and value for any calculated variable.

## Tasks:

Map the values from the GIS data to the standard list of locations.

questions or comments: analyst@hammerdirt.ch

In [1]:
# sys, file and nav packages:
import os
import datetime as dt
import csv
import json

# math packages:
import pandas as pd
import numpy as np

# home brew utitilties
import resources.utility_functions as ut

# documenting
from IPython.display import Markdown as md

### Get and merge data

The GIS data is calculated as the locations are added to the standard list. There are several lists that need to be merged. They are stored in the directory _gislanduse_. 

In [2]:
today = dt.datetime.now().date().strftime("%Y-%m-%d")

# beach data
dfBeaches = pd.read_csv("resources/beaches_pop_bfs.csv")

# key location to slug:
location_slug_map = dfBeaches[['location', 'slug']].set_index('location')

# the geo data needs to be collected from the gislanduse directory
# this is qgis output with aggregated values for meters of streets and number of river intersects
# within 1500 meters of the survey location
geo_data = pd.read_csv("add_exp_variables/gislanduse/beaches_no_buildings_data.csv")
alpes_data = pd.read_csv("add_exp_variables/gislanduse/alpes_valaisanees_streets.csv")
alpes_data2 = pd.read_csv("add_exp_variables/gislanduse/alpes_streets.csv")
# the different sources need to be combined into one structure
geo_data = geo_data[['slug','intersects', 'length']].copy()
geo_data.head()

,slug,intersects,length
0,aare_solothurn_nguyena,0,94192
1,les-glariers,0,106984
2,mullermatte,3,109589
3,murg-bad,6,42890
4,tiger-duck-beach,1,105251


In [3]:
alpes_data.drop("ERSTELL_J", axis=1, inplace=True)
alpes_data.rename(columns={'streets':'length'}, inplace=True)
alpes_data = alpes_data[['slug','intersects', 'length']]
alpes_data.head()

,slug,intersects,length
0,clean-up-tour-crans-montana,0,44197.553
1,clean-up-tour-nendaz,0,49738.888
2,clean-up-tour-morgins,0,66409.626
3,clean-up-tour-veysonnaz,0,65355.751
4,clean-up-tour-les-crosets,0,50272.353


In [4]:
alpes_data2.rename(columns={'streeets':'length'}, inplace=True)
alpes_data2 = alpes_data2[['slug','intersects', 'length']]
alpes_data2.head()

,slug,intersects,length
0,clean-up-tour-san-bernardino,0,43273
1,clean-up-tour-la-robella,0,52625
2,clean-up-tour-andermatt,0,22019
3,clean-up-tour-la-tzoumaz,0,74080
4,clean-up-tour-grindelwald,0,34188


In [5]:
# merge the new data
geo_data = geo_data.append(alpes_data)
geo_data = geo_data.append(alpes_data2)
geo_data.rename(columns={'length':'streets'}, inplace=True)
geo_data.set_index('slug', inplace=True)

# make a map for each variable of interest
streets = geo_data['streets']
intersects = geo_data['intersects']

### Map Streets and intersects to beach data

When the attributes surounding each survey location are calculated in QGIS the data is keyed to the survey location 'slug'. Therefore the 'slug' column of the beach data can be used to map the results from the GIS data to the beach data.

In [6]:
# a function to tag records that do not have any corresponding GIS data
def check_values(df, x):
    try:
        package = df.loc[x]
    except:
        package =0
    return package

# check each variable
dfBeaches['streets'] = dfBeaches.slug.map(lambda x:check_values(streets, x,))
dfBeaches['streets'] = dfBeaches.streets.astype('int')
no_streets = dfBeaches[dfBeaches.streets == 0]

dfBeaches['intersects'] = dfBeaches.slug.map(lambda x:check_values(intersects, x))
no_intersects = dfBeaches[dfBeaches.intersects == 'none']

# collect the locations that had no gis data
no_geo_data = list(set(no_intersects.slug.unique()) | set(no_streets.slug.unique()))

print(F"These are the locations that have no intersects data:\n")
print(F"{no_intersects['slug'].values}\n")
print(F"These are the locations that have no streets data:\n")
print(F"{no_streets.slug.values}\n")

These are the locations that have no intersects data:

[]

These are the locations that have no streets data:

['sihl_horgen_bucherf']



#### Export the data

In [7]:
filename = "add_exp_variables/beaches_with_intersects_and_streets.csv"
dfBeaches.to_csv(filename, index=False)

In [8]:
author = "roger@hammerdirt.ch"
my_message = "Love what you do. \u2764\ufe0f"
md(F"""<br></br>
**This project was made possible by the Swiss federal office for the environment.**<br>

>{my_message}<br>

*{author}* pushed the run button on {today}.<br>
This document originates from https://github.com/hammerdirt-analyst/IQAASL-End-0f-Sampling-2021 all copyrights apply.<br>
""")

<br></br>
**This project was made possible by the Swiss federal office for the environment.**<br>

>Love what you do. ❤️<br>

*roger@hammerdirt.ch* pushed the run button on 2021-09-19.<br>
This document originates from https://github.com/hammerdirt-analyst/IQAASL-End-0f-Sampling-2021 all copyrights apply.<br>
